# Querying the Task Dataset


## Observational Data:

In [3]:
# encoding=utf8
import os
import numpy as np
import pandas as pd
import getpass
import pickle
import string
import matplotlib.pyplot as plt
%matplotlib inline

# Point this at the ONET data: 
datasets = '/home/'+ getpass.getuser() +'/Datasets/ONET/'
# datasets = '/Users/Logan/Google Drive/Oxford/DPhil/projects/past/future_employment/data/'
print(datasets)

/home/scpd/Datasets/ONET/


In [33]:
occupations = pd.read_table(os.path.join(datasets, 'databases/db2016/Occupation Data.txt'), sep='\t')
occupations.rename(columns = {'Title':'Occupation title'}, inplace = True)
occupations.head()

,O*NET-SOC Code,Occupation title,Description
0,11-1011.00,Chief Executives,Determine and formulate policies and provide overall direction of companies or private and public sector organizations within guidelines set up by a board of directors or similar governing body. P...
1,11-1011.03,Chief Sustainability Officers,"Communicate and coordinate with management, shareholders, customers, and employees to address sustainability issues. Enact or oversee a corporate sustainability strategy."
2,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of public or private sector organizations. Duties and responsibilities include formulating policies, managing daily operations, and planning the use of m..."
3,11-1031.00,Legislators,"Develop, introduce or enact laws and statutes at the local, tribal, State, or Federal level. Includes only workers in elected positions."
4,11-2011.00,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising policies and programs or produce collateral materials, such as posters, contests, coupons, or give-aways, to create extra interest in the purchase of a prod..."


In [5]:
tasks = pd.read_table(os.path.join(datasets, 'databases/db2016/Task Statements.txt'), sep='\t')

# tasks = tasks[['O*NET-SOC Code', 'Task ID', 'Task']]
reduced_tasks = tasks[tasks['O*NET-SOC Code'].isin(tasks['O*NET-SOC Code'].unique())]

# All tasks are unique to Occupation it seems
print(tasks.shape, tasks['O*NET-SOC Code'].unique().shape, reduced_tasks.shape)
tasks.head()

(19566, 7) (974,) (19566, 7)


,O*NET-SOC Code,Task ID,Task,Task Type,Incumbents Responding,Date,Domain Source
0,11-1011.00,8823,Direct or coordinate an organization's financi...,Core,87.0,07/2014,Incumbent
1,11-1011.00,8831,Appoint department heads or managers and assig...,Core,87.0,07/2014,Incumbent
2,11-1011.00,8825,Analyze operations to evaluate performance of ...,Core,87.0,07/2014,Incumbent
3,11-1011.00,8826,"Direct, plan, or implement policies, objective...",Core,87.0,07/2014,Incumbent
4,11-1011.00,8827,"Prepare budgets for approval, including those ...",Core,87.0,07/2014,Incumbent


In [28]:
#Task DWAs (detailed work activitiy code):
taskDWA = pd.read_table(os.path.join(datasets, 'databases/db2016/Tasks to DWAs.txt'), sep='\t')
taskDWA = taskDWA[['Task ID', 'DWA ID']]

print(taskDWA.shape, "UNIQUE DWA: ", len(taskDWA['DWA ID'].unique()))
#Tasks to DWAs are many-to-many
taskDWA.head()

(22838, 2) UNIQUE DWA:  2070


,Task ID,DWA ID
0,20461,4.A.2.a.4.I09.D03
1,20461,4.A.4.b.6.I08.D04
2,8823,4.A.4.b.4.I09.D02
3,8824,4.A.4.a.2.I03.D14
4,8825,4.A.2.a.4.I07.D09


In [45]:
df = pd.merge(tasks, taskDWA,  how='left', left_on=['Task ID'], right_on = ['Task ID']).sort_values(by = 'Task ID')
df = df[df['DWA ID'].notnull()]
df['IWA ID'] = df['DWA ID'].str.slice(0,-4)    # create IWA ID
df['WA ID'] = df['DWA ID'].str.slice(0,-8)     # create WA ID

## ADD DWA and IWA titles:
DWAref = pd.read_table(os.path.join(datasets, 'databases/db2016/DWA Reference.txt'), sep='\t')[['DWA ID', 'DWA Title']]
taskDWA2 = pd.merge(df, DWAref,  how='left', left_on=['DWA ID'], right_on = ['DWA ID'])

IWAref = pd.read_table(os.path.join(datasets, 'databases/db2016/IWA Reference.txt'), sep='\t')[['IWA ID', 'IWA Title']]
df2 = pd.merge(taskDWA2, IWAref,  how='left', left_on=['IWA ID'], right_on = ['IWA ID'])

# cols = ['Task ID', 'Task', 'DWA ID', 'DWA Title', 'IWA ID', 'IWA Title', 'WA ID']
cols = ['O*NET-SOC Code', 'Task ID', 'DWA ID', 'IWA ID', 'WA ID']
# df2 = df2[cols]

# tasks are many-to-many with DWA, e.g. task id=8826.
print(df2.shape, "UNIQUE DWA: ", len(df2['DWA ID'].unique()))

(22838, 12) UNIQUE DWA:  2070


In [30]:
pd.set_option('max_colwidth',200)

df2[df2["DWA ID"] == "4.A.4.b.4.I09.D08"]

,O*NET-SOC Code,Task ID,Task,Task Type,Incumbents Responding,Date,Domain Source,DWA ID,IWA ID,WA ID,DWA Title,IWA Title
11840,21-2011.00,10949,Collaborate with committees or individuals to address financial or administrative issues pertaining to congregations.,Core,88.0,07/2015,Incumbent,4.A.4.b.4.I09.D08,4.A.4.b.4.I09,4.A.4.b.4,Manage organizational or program finances.,Manage budgets or finances.
11841,21-2011.00,10951,"Perform administrative duties, such as overseeing building management, ordering supplies, contracting for services or repairs, or supervising the work of staff members or volunteers.",Core,88.0,07/2015,Incumbent,4.A.4.b.4.I09.D08,4.A.4.b.4.I09,4.A.4.b.4,Manage organizational or program finances.,Manage budgets or finances.
14094,21-2021.00,12986,Analyze revenue and program cost data to determine budget priorities.,Core,94.0,07/2015,Incumbent,4.A.4.b.4.I09.D08,4.A.4.b.4.I09,4.A.4.b.4,Manage organizational or program finances.,Manage budgets or finances.
22030,21-1015.00,20234,"Manage budgets and direct case service allocations, authorizing expenditures and payments.",Supplemental,115.0,07/2012,Incumbent,4.A.4.b.4.I09.D08,4.A.4.b.4.I09,4.A.4.b.4,Manage organizational or program finances.,Manage budgets or finances.


In [37]:
for soc in df2[df2["DWA ID"] == "4.A.4.b.4.I09.D08"]["O*NET-SOC Code"].unique():
    print(soc, occupations[occupations["O*NET-SOC Code"] == soc]["Occupation title"].unique())
    print(df2[df2["O*NET-SOC Code"] == soc]["DWA Title"].unique())

21-2011.00 ['Clergy']
['Lead classes or community events.'
 'Counsel clients or patients regarding personal issues.'
 'Visit individuals in their homes to provide support or information.'
 'Train staff members in social services skills.'
 'Interpret cultural or religious information for others.'
 'Intervene in crisis situations to assist clients.'
 'Develop promotional strategies for religious organizations.'
 'Manage organizational or program finances.'
 'Refer clients to community or social service programs.'
 'Develop educational programs.'
 'Plan conferences, programs, or special events.']
21-2021.00 ['Directors, Religious Activities and Education']
['Supervise workers providing client or patient services.'
 'Train staff members in social services skills.'
 'Lead classes or community events.' 'Develop educational programs.'
 'Plan conferences, programs, or special events.'
 'Advise clients or community groups on health issues.'
 'Counsel clients regarding interpersonal issues.'
 'C

In [47]:
df2[df2["WA ID"] == "4.A.4.b.4"]["DWA Title"].unique()

array(['Direct sales, marketing, or customer service activities.',
       'Prepare operational budgets.',
       'Manage human resources activities.', 'Supervise employees.',
       'Direct administrative or support services.',
       'Manage construction activities.',
       'Direct facility maintenance or repair activities.',
       'Direct organizational operations, projects, or services.',
       'Manage control system activities in organizations.',
       'Coordinate special events or programs.',
       'Direct financial operations.',
       'Prepare staff schedules or work assignments.',
       'Manage operations, research, or logistics projects.',
       'Coordinate training activities.',
       'Coordinate personnel recruitment activities.',
       'Supervise engineering or other technical personnel.',
       'Direct design or development activities.',
       'Supervise scientific or technical personnel.',
       'Manage agricultural or forestry operations.',
       'Direct nat